# The Need for Efficient API Calls

Problem 1: Interrupted Workflow/Kernel.

Problem 2: Unexpected Errors

Problem 3: Is my loop working??

Problem 4: Overwriting Previous Results.

Problem 5: Duplicating API calls



##Solutions:
With some foresight and planning, we can build in protections against each of these potential problems.

The Safeguards to Add
- Create your empty results file ONLY if it doesn't already exist.
- Save your API results within your for loop so you always have the result of EVERY API call saved to disk.
- Add a progress bar to your for loop so you can confirm that its running and estimate how much longer until it is done.
- Double-check your list of API calls against your previously saved results and remove any API calls that you already have the data for.

## Adding Safeguards to our Data Extraction Workflow


In [3]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [4]:
# Install tmdbsimple (only need to run once)
!pip install tqdm

### Load API Credentials

In [7]:
# Load API Credentials
with open('/Users/dmburns/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [8]:
# set our API call parameters 
LOCATION = 'NY,NY'
TERM = 'Pizza'

In [9]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_NY_pizza.json"
JSON_FILE

'Data/results_in_progress_NY_pizza.json'

In [10]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_NY_pizza.json not found. Saving empty list to file.


In [11]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [12]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [13]:
## How many results total?
total_results = results['total']
total_results

12200

In [14]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [15]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

610

In [16]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [17]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [18]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/610 [00:00<?, ?it/s]

In [19]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)

  0%|          | 0/610 [00:00<?, ?it/s]

YelpAPIError: VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.

Two ways to delete json file we just created

In [20]:
## delete file and confirm it no longer exits.
os.remove(JSON_FILE)
os.path.isfile(JSON_FILE)

False

In [21]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [22]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_NY_pizza.json not found. Saving empty list to new file.
- 0 previous results found.


610

In [23]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/610 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [24]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,zj8Lq1T8KIC5zwFief15jg,prince-street-pizza-new-york-2,Prince Street Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/I4gm7i...,False,https://www.yelp.com/biz/prince-street-pizza-n...,4662,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.72308755605564, 'longitude': -...","[delivery, pickup]",$,"{'address1': '27 Prince St', 'address2': None,...",+12129664100,(212) 966-4100,1961.877142
1,ysqgdbSrezXgVwER2kQWKA,julianas-brooklyn-3,Juliana's,https://s3-media2.fl.yelpcdn.com/bphoto/NVoLFl...,False,https://www.yelp.com/biz/julianas-brooklyn-3?a...,2585,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.70274718768062, 'longitude': -...",[delivery],$$,"{'address1': '19 Old Fulton St', 'address2': '...",+17185966700,(718) 596-6700,308.569844
2,WG639VkTjmK5dzydd1BBJA,rubirosa-new-york-2,Rubirosa,https://s3-media4.fl.yelpcdn.com/bphoto/LuSzR8...,False,https://www.yelp.com/biz/rubirosa-new-york-2?a...,2847,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.722766, 'longitude': -73.996233}",[pickup],$$,"{'address1': '235 Mulberry St', 'address2': ''...",+12129650500,(212) 965-0500,1932.946770
3,v1DHGRNCH9247WLYoaoA9A,l-industrie-pizzeria-brooklyn,L'industrie Pizzeria,https://s3-media2.fl.yelpcdn.com/bphoto/SkMxUE...,False,https://www.yelp.com/biz/l-industrie-pizzeria-...,838,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.71162, 'longitude': -73.95783}",[delivery],$,"{'address1': '254 S 2nd St', 'address2': '', '...",+17185990002,(718) 599-0002,3145.016041
4,WIhm0W9197f_rRtDziq5qQ,lombardis-pizza-new-york-4,Lombardi's Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/UZ6V_h...,False,https://www.yelp.com/biz/lombardis-pizza-new-y...,6492,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.7215934960083, 'longitude': -7...","[delivery, pickup]",$$,"{'address1': '32 Spring St', 'address2': '', '...",+12129417994,(212) 941-7994,1798.995978


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,xmNxfsWLBZlUw2tyoiaN-g,tornas-pizzeria-hoboken,Torna's Pizzeria,https://s3-media4.fl.yelpcdn.com/bphoto/H1hLIt...,False,https://www.yelp.com/biz/tornas-pizzeria-hobok...,89,"[{'alias': 'pizza', 'title': 'Pizza'}]",3.5,"{'latitude': 40.7479313, 'longitude': -74.0307...","[pickup, delivery]",$,"{'address1': '254 9th St', 'address2': '', 'ad...",+12017988873,(201) 798-8873,5644.238790
996,Zub6WrbrS_4XMI1mPbyeiw,panineria-italiana-new-york,Panineria Italiana,https://s3-media3.fl.yelpcdn.com/bphoto/Gd5r1e...,False,https://www.yelp.com/biz/panineria-italiana-ne...,268,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 40.73211, 'longitude': -73.99569}","[pickup, delivery]",$$,"{'address1': '9 East 8th St', 'address2': None...",+19173883257,(917) 388-3257,2970.617415
997,gED3ep4dX5FP_FYVTkokIQ,salvatores-of-soho-bronx,Salvatores of Soho,https://s3-media3.fl.yelpcdn.com/bphoto/aqrp4F...,False,https://www.yelp.com/biz/salvatores-of-soho-br...,331,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 40.8875232, 'longitude': -73.90704}",[delivery],$$,"{'address1': '3738 Riverdale Ave', 'address2':...",+17185487499,(718) 548-7499,21553.151257
998,M6Mb4kEmv0FF3uM9Z0ONCA,marios-pizza-brooklyn-3,Mario's Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/BcI_wA...,False,https://www.yelp.com/biz/marios-pizza-brooklyn...,39,"[{'alias': 'pizza', 'title': 'Pizza'}]",3.5,"{'latitude': 40.62488, 'longitude': -73.99929}","[pickup, delivery]",$,"{'address1': '6424 14th Ave', 'address2': '', ...",+17182325440,(718) 232-5440,8964.822129
999,nab2tXtO23_s9fb2WdksdA,luna-pizza-brooklyn,Luna Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/SezhPR...,False,https://www.yelp.com/biz/luna-pizza-brooklyn?a...,20,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.60773, 'longitude': -73.961341}",[delivery],$,"{'address1': '1115 Kings Hwy', 'address2': '',...",+17189984400,(718) 998-4400,11216.240927


In [ ]:
# check for duplicate results
final_df.duplicated().sum()

Because our yelp results include columns that contain lists, we cannot check every column in dataframe for duplicates.
Instead, we can use the subset argument for df.duplicated() and df.drop_duplicates() to only check the id column for duplicates.

In [25]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

118

In [26]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [27]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_NY_pizza.csv.gz', compression='gzip',index=False)